In [1]:
import setup
setup.init_django()
from market import services as market_services
from market import tasks as market_tasks
from market.models import Company
import json
from decouple import config

In [2]:
ticker = "GOOG"
name = "Google"
company, _ = Company.objects.get_or_create(name=name, ticker=ticker)
company.id

3

In [ ]:
market_tasks.sync_historical_stock_data(
    years_ago=1, 
    company_ids=[company.id], 
    use_celery=False, 
    verbose=True
)

# use celery / async
# market_tasks.sync_historical_stock_data.delay(years_ago=5, company_ids=[company.id], use_celery=True, verbose=False)

In [4]:
days = 90
queryset = market_services.get_daily_stock_quotes_queryset(ticker, days=days)

In [5]:
averages = market_services.get_daily_moving_averages(ticker, days=days, queryset=queryset)
price_target = market_services.get_price_target(ticker, days=days, queryset=queryset)
volume_trend = market_services.get_volume_trend(ticker, days=days, queryset=queryset)
rsi_data = market_services.calculate_rsi(ticker, days=days, period=14)
macd_data = market_services.calculate_macd(ticker, days=days, queryset=queryset)
bollinger_data = market_services.calculate_bollinger_bands(ticker, days=days, queryset=queryset)

In [6]:
signals = []
signal_weights = {
    'ma_crossover': 2.0,    
    'price_target': 1.5,   
    'volume': 1.0,         
    'rsi': 1.5,          
    'macd': 1.5,          
    'bollinger': 1.0       
}

In [7]:
if averages.get('ma_5') > averages.get('ma_20'):
    signals.append(1 * signal_weights['ma_crossover'])
else:
    signals.append(-1 * signal_weights['ma_crossover'])

In [8]:
if price_target.get('current_price') < price_target.get('conservative_target'):
    signals.append(1 * signal_weights['price_target'])
else:
    signals.append(-1 * signal_weights['price_target'])

In [9]:
vol_change = volume_trend.get("volume_change_percent")
if vol_change > 20:
    signals.append(1 * signal_weights['volume'])
elif vol_change < -20:
    signals.append(-1 * signal_weights['volume'])
else:
    signals.append(0)

In [10]:
rsi = rsi_data.get('rsi')
if rsi > 70:
    signals.append(-1 * signal_weights['rsi'])  # Overbought
elif rsi < 30:
    signals.append(1 * signal_weights['rsi'])   # Oversold
else:
    signals.append(0)

In [11]:
if macd_data:
    if macd_data['macd_line'] > 0:
        signals.append(1 * signal_weights['macd'])
    else:
        signals.append(-1 * signal_weights['macd'])

In [12]:
if bollinger_data:
    current_price = price_target.get('current_price')
    if current_price < bollinger_data['lower_band']:
        signals.append(1 * signal_weights['bollinger'])
    elif current_price > bollinger_data['upper_band']:
        signals.append(-1 * signal_weights['bollinger'])
    else:
        signals.append(0)

In [13]:
weighted_score = sum(signals)
max_possible_score = sum(signal_weights.values())
normalized_score = (weighted_score / max_possible_score) * 10

In [14]:
if normalized_score >= 3:
    recommendation = "BUY"
elif normalized_score <= -3:
    recommendation = "SELL"
else:
    recommendation = "HOLD"

In [15]:
analysis_result = {
    "score": round(normalized_score, 2),
    "recommendation": recommendation,
    "ticker": ticker,
    "indicators": {
        **averages,
        **price_target,
        **volume_trend,
        **rsi_data,
        "macd": macd_data,
        "bollinger": bollinger_data
    }
}

print(json.dumps(analysis_result, indent=2))

{
  "score": 4.71,
  "recommendation": "BUY",
  "ticker": "GOOG",
  "indicators": {
    "ma_5": 194.424,
    "ma_20": 194.0835,
    "current_price": 197.15,
    "conservative_target": 210.0769,
    "aggressive_target": 218.0631,
    "average_price": 182.3596,
    "avg_volume": 1582.123287671233,
    "latest_volume": 1015,
    "volume_change_percent": -35.84570760638989,
    "rsi": 61.4011,
    "avg_gain": 1.5733,
    "avg_loss": 0.9891,
    "period": 14,
    "days": 90,
    "macd": {
      "macd_line": 2.024502389854746,
      "signal_line": 2.3549156133238993,
      "histogram": -0.33041322346915347
    },
    "bollinger": {
      "middle_band": 194.0835,
      "upper_band": 198.9751,
      "lower_band": 189.1919
    }
  }
}
